In [1]:
import numpy as np
import pandas as pd
from pqp.data.data import Data
from pqp.estimation.model import Model
from pqp.causal.graph import Graph
from pqp.parametric.categorical_distribution import CategoricalDistribution
from pqp.causal.estimands import *
from pqp.symbols import *

In [2]:
x, y, z = make_vars("xyz")
df = pd.DataFrame({
    "x": [0, 1, 0, 1, 1.5, 1.5],
    "y": [0, 1, 1, 2, 1.5, 2.5],
    "m": [1, 2, 3, 4, 5, 6],
    "z": [0, 0, 1, 1, 0, 1],
})
data = Data(df)
print(data.domains())

dist = CategoricalDistribution(data, prior=0.1)
data.domains()

{Variable(x): RealDomain([0.0, 1.5]), Variable(y): RealDomain([0.0, 2.5]), Variable(m): IntegerDomain([1, 6]), Variable(z): BinaryDomain()}


{Variable(x): CategoricalDomain(cardinality=2),
 Variable(y): CategoricalDomain(cardinality=2),
 Variable(m): IntegerDomain([1, 6]),
 Variable(z): BinaryDomain()}

In [3]:
parametric_model = CategoricalDistribution(data, prior=0.1)
causal_model = Graph([
    y <= [x, z],
    x <= z,
    ])

In [4]:
df.x

0    (-0.0015, 0.75]
1        (0.75, 1.5]
2    (-0.0015, 0.75]
3        (0.75, 1.5]
4        (0.75, 1.5]
5        (0.75, 1.5]
Name: x, dtype: category
Categories (2, interval[float64, right]): [(-0.0015, 0.75] < (0.75, 1.5]]